## YLOL
### PyTorch 기반 물체인식 모델
- CNN, rCNN(Regions with CNN)
- https://github.com/ultralytics/ultralytics 참조

#### YOLOv5 이상 설치
```shell
    > pip install
```

In [5]:
# YOLO 설치
!pip install ultralytics


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 콘솔에서 예측

In [6]:
# 콘솔에서 예측
## yolo11n.pt - pretrained YOLO model
## 자동으로 yolo11n.pt 다운로드
## 웹 URL에 있는 이미지도 예측 가능
!yolo predict model=yolo11n.pt source='https://www.ultralytics.com/images/bus.jpg'

Ultralytics 8.3.109 🚀 Python-3.11.9 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

Found https://www.ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 c:\Source\iot-dataanalysis-2025\day08\bus.jpg: 640x480 4 persons, 1 bus, 31.6ms
Speed: 2.3ms preprocess, 31.6ms inference, 58.1ms postprocess per image at shape (1, 3, 640, 480)
Results saved to c:\Source\iot-dataanalysis-2025\runs\detect\predict8
💡 Learn more at https://docs.ultralytics.com/modes/predict


#### 파이썬으로 예측

In [7]:
# YOLO 모듈 로드
from ultralytics import YOLO

In [8]:
# YOLO 클래스가 들어오는 모델의 버전에 따라 알아서 YOLO 예측모델 객체 생성
model = YOLO('./yolo11n.pt')

##### coco8.ymal
- https://github.com/ultralytics/assets/releases/download/v0.0.0/coco8.zip
- 위 내용대로 훈련을 시킨 결과 -> yolo11n.pt

In [ ]:
# coco8.ymal - YOLO 훈련에 사용할 데이터셋 정의파일
train_results = model.train(
    data = './coco8.yaml',
    epochs = 100,
    imgsz = 640,
    device = 'cuda:0'
)

### 이미지 예측

In [10]:
result = model('./01.jpg')


image 1/1 c:\Source\iot-dataanalysis-2025\day08\01.jpg: 480x640 1 cat, 34.0ms
Speed: 1.5ms preprocess, 34.0ms inference, 80.7ms postprocess per image at shape (1, 3, 480, 640)


In [11]:
# matplotlib 모듈 로드
import matplotlib.pyplot as plt
from PIL import Image

In [12]:
# 예측결과 이미지 저장
img = result[0].plot()
img_pil = Image.fromarray(img[..., ::-1])
img_pil.save('./predict_result.jpg')

### OpenCV
- Open source Computer Vision 약자. 실시간 컴퓨터 비전(시각처리)을 목적으로 프로그래밍 라이브러리
- 인텔에서 2000년 C, C++ 에서 사용하기 위해서 개발
- 파이썬에서 사용할 수 있게 래핑
```shell
> pip install opencv-python

In [13]:
# openCV 설치
!pip install opencv-python


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import cv2
cv2.__version__

'4.11.0'

In [15]:
img2 = cv2.imread('./predict_result.jpg')
img2.shape

(464, 640, 3)

In [16]:
# 윈도우 창 오픈
cv2.imshow('predict_result', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### YOLO 예측

In [17]:
img = cv2.imread('./02.jpg')
resized_img = cv2.resize(img, (640, 400))

result = model(resized_img)
plots = result[0].plot()

cv2.imshow('predict_result', plots)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 416x640 1 bottle, 1 cup, 1 bowl, 1 tv, 1 mouse, 1 book, 165.1ms
Speed: 15.5ms preprocess, 165.1ms inference, 2.2ms postprocess per image at shape (1, 3, 416, 640)


#### 동영상 플레이
- 라즈베리파이에서 동일하게 사용가능
- 라즈베리파이 웹캡 사용추천

In [18]:
# 비디오 파일 경로
video_path = './sample01.mp4'
output_path = './sample01_output.mp4'
count_path = './sample01_count.mp4'

In [19]:
# 동영상 플레이
cap = cv2.VideoCapture(video_path) # 0 -> 웹캡이나 카메라 설치된 번호

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    cv2.imshow('Video play', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): # q버튼을 누르면
        break

cap.release() # 비디오를 해제
cv2.destroyAllWindows()

#### YOLO 실시간 예측

In [20]:
import time

In [21]:
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS) # 동영상 FPS(Frame Per Second)
frame_time = 1.0 / fps # 초단위로 변환
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # 1280
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 720

# VideoWriter 객체 생성(동영상 화면에 그림, 글자를 그리기 위한 객체)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    start_time = time.time() # 시작시간
    ret, frame = cap.read()
    if not ret: break

    # 객체 탐지
    results = model(frame)
    # 탐지 결과 그리기
    for result in results:
        detect_frame = result.plot()
    # 결과프레임을 파일로 저장
    out.write(detect_frame)
    # 결과 표시
    cv2.imshow('YOLO Object Detection', detect_frame)
    cv2.imshow('Video play', frame)

    # 프레임간 실제 지연시간 계산
    elapsed_time = time.time() - start_time
    delay = max(int((frame_time - elapsed_time) * 1000), 1)

    if cv2.waitKey(1) & 0xFF == ord('q'): # q버튼을 누르면
        break

cap.release() # 비디오를 해제
cap = cv2.VideoCapture(video_path) 

fps = cap.get(cv2.CAP_PROP_FPS)   # 동영상 FPS(Frame Per Second)
frame_time = 1.0 /fps  # 초단위로 변환
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))      # 1280
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))    # 720

# VideoWriter 객체 생성(동영상 화면에 그림, 글자를 그리기 위한 객체)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    start_time = time.time() # 시작시간 
    ret, frame = cap.read()
    if not ret: break

    # 객체 탐지
    results = model(frame)
    # 탐지 결과 그리기
    for result in results:
        detect_frame = result.plot()
    # 결과프레임을 파일로 저장
    out.write(detect_frame)
    # 결과 표시
    cv2.imshow('YOLO Object Detection', detect_frame)
    cv2.imshow('Video play', frame)

    # 프레임간 실제 지연시간 계산
    elapsed_time = time.time() - start_time
    delay = max(int((frame_time  - elapsed_time) * 1000), 1)

    if cv2.waitKey(1) & 0xFF == ord('q'): # q버튼을 누르면
        break

cap.release()  # 비디오를 해제
out.release()
cv2.destroyAllWindows()


0: 384x640 1 train, 86.4ms
Speed: 4.2ms preprocess, 86.4ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 train, 149.7ms
Speed: 3.9ms preprocess, 149.7ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 train, 17.3ms
Speed: 6.5ms preprocess, 17.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 train, 11.0ms
Speed: 1.3ms preprocess, 11.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 train, 12.0ms
Speed: 1.5ms preprocess, 12.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 train, 11.7ms
Speed: 1.5ms preprocess, 11.7ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 train, 11.3ms
Speed: 1.5ms preprocess, 11.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 train, 10.7ms
Speed: 2.2ms preprocess, 10.7ms inference, 1.5ms postprocess per image at 

#### Car Counting
- 지정된 라인 아래로 내려오는 자동차 개수 카운팅

- shapely
```shell
    !pip install shapely==2.0.1
```

In [22]:
# shapely 설치
!pip install shapely==2.0.1


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
# lap 설치
!pip install lap


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import cv2
from ultralytics.solutions import ObjectCounter

In [25]:
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS) # 동영상 FPS(Frame Per Second)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # 1280
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 720

# 지정된 라인
region_points = [(20, 400), (1080, 400)]

# VideoWriter 객체 생성(동영상 화면에 그림, 글자를 그리기 위한 객체)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(count_path, fourcc, fps, (width, height))

# 물체 인식 객체
counter = ObjectCounter(
    show=True, # 처리하는 동안 디스플레이  여부
    region=region_points, # 카운팅할 위치
    model='./yolo11n.pt', # YOLO11 모델
    line_width=3, # 라인 두께
)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    results = counter(frame)
    out.write(results.plot_im)

cap.release() # 비디오를 해제
out.release()
cv2.destroyAllWindows()

Ultralytics Solutions:  {'region': [(20, 400), (1080, 400)], 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'records': 5, 'show': True, 'model': './yolo11n.pt', 'line_width': 3}

0: 384x640 1 train, 12.3ms
Speed: 1.8ms preprocess, 12.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
 Results: SolutionResults(classwise_count={'train': {'IN': 0, 'OUT': 0}}, total_tracks=1)

0: 384x640 1 train, 11.1ms
Speed: 2.4ms preprocess, 11.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
 Results: SolutionResults(classwise_count={'train': {'IN': 0, 'OUT': 0}}, total_tracks=1)

0: 384x640 1 train, 15.5ms
Speed: 1.7ms preprocess, 15.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
 Results: SolutionResults(classwise_count={'train': {'IN': 0, 'OUT': 0}}, total_tracks=1)

0: 384x640 1 train, 11.1ms
Speed: 1.5ms preprocess, 11.1ms inference